In [42]:
from nba_api.stats.static import teams as _teams

nba_teams = _teams.get_teams()
temp = _teams.find_teams_by_full_name('lakers')
nba_teams_name = [team['full_name'] for team in nba_teams]
nba_teams_id = [team['id'] for team in nba_teams]
print(temp)
print(nba_teams_id)

[{'id': 1610612747, 'full_name': 'Los Angeles Lakers', 'abbreviation': 'LAL', 'nickname': 'Lakers', 'city': 'Los Angeles', 'state': 'California', 'year_founded': 1948}]
[1610612737, 1610612738, 1610612739, 1610612740, 1610612741, 1610612742, 1610612743, 1610612744, 1610612745, 1610612746, 1610612747, 1610612748, 1610612749, 1610612750, 1610612751, 1610612752, 1610612753, 1610612754, 1610612755, 1610612756, 1610612757, 1610612758, 1610612759, 1610612760, 1610612761, 1610612762, 1610612763, 1610612764, 1610612765, 1610612766]


In [53]:
import pandas as pd

teams = pd.read_csv('teams.csv')
for index, row in teams.iterrows():
    from_year  = row['From']
    to_year = row['To']
    two_num = from_year[2:4]
    two_num2 = to_year[5:7]
    # Assign new year to the row
    if (from_year[0] == "1"):
        teams.at[index, 'From'] = int("19{}".format(two_num))
    elif (from_year[0] == "2"):
        teams.at[index, 'From'] = int("20{}".format(two_num))
        
    if (to_year[0] == "1"):
        teams.at[index, 'To'] = int("19{}".format(two_num2))
    elif (to_year[0] == "2"):
        teams.at[index, 'To'] = int("20{}".format(two_num2))
        
# Remove teams that have a to year of less than 1980
teams = teams[teams['To'] >= 1980]
# Keep all Lg == NBA or NBA/BAA
teams = teams[teams['Lg'].str.contains('NBA')]

for index, row in teams.iterrows():
    if (row['Franchise'] not in nba_teams_name):
        teams.drop(index, inplace=True)
        
unique_teams = list(teams['Franchise'].unique())
for team in unique_teams:
    entries_current_team = teams[teams['Franchise'] == team]
    # Find the entry with the lowest year
    min_year = entries_current_team['From'].min()
    max_year = entries_current_team['To'].max()
    # Keep all other entries
    entries_current_team2 = entries_current_team[entries_current_team['From'] != min_year]
    for index, row in entries_current_team2.iterrows():
        teams.drop(index, inplace=True)
    entries_current_team2 = entries_current_team[entries_current_team['To'] != max_year]
    for index, row in entries_current_team2.iterrows():
        teams.drop(index, inplace=True)
    
# Add the team id to the dataframe
for index, row in teams.iterrows():
    temp = _teams.find_teams_by_full_name(row['Franchise'])
    teams.at[index, 'TeamID'] = temp[0]['id']
    
# Make the TeamID column an integer and first column in the dataframe
teams['TeamID'] = teams['TeamID'].astype(int)
columns = teams.columns.tolist()
columns = columns[-1:] + columns[:-1]
teams = teams[columns]

# Reindex the dataframe
teams = teams.reset_index(drop=True)

# Drop the Active column
teams.drop('Active', axis=1, inplace=True)
print(teams)

        TeamID               Franchise       Lg  From    To  Yrs     G     W  \
0   1610612737           Atlanta Hawks      NBA  1949  2024   75  5878  2900   
1   1610612738          Boston Celtics  NBA/BAA  1946  2024   78  6054  3587   
2   1610612751           Brooklyn Nets  NBA/ABA  1967  2024   57  4553  2009   
3   1610612766       Charlotte Hornets      NBA  1988  2024   34  2653  1160   
4   1610612741           Chicago Bulls      NBA  1966  2024   58  4623  2353   
5   1610612739     Cleveland Cavaliers      NBA  1970  2024   54  4295  1997   
6   1610612742        Dallas Mavericks      NBA  1980  2024   44  3484  1762   
7   1610612743          Denver Nuggets  NBA/ABA  1967  2024   57  4556  2326   
8   1610612765         Detroit Pistons  NBA/BAA  1948  2024   76  5940  2815   
9   1610612744   Golden State Warriors  NBA/BAA  1946  2024   78  6044  2933   
10  1610612745         Houston Rockets      NBA  1967  2024   57  4545  2340   
11  1610612754          Indiana Pacers  

In [58]:
nba_teams = _teams.get_teams()
abbreviations = [(team['abbreviation'], team['id']) for team in nba_teams]

for index, row in teams.iterrows():
    for team in abbreviations:
        if (row['TeamID'] == team[1]):
            # Add column with abbreviation
            teams.at[index, 'Abbreviation'] = team[0]
            
# Make abbreviation the Third column
columns = teams.columns.tolist()
columns = columns[:2] + columns[-1:] + columns[2:-1]
teams = teams[columns]

        TeamID               Franchise Abbreviation       Lg  From    To  Yrs  \
0   1610612737           Atlanta Hawks          ATL      NBA  1949  2024   75   
1   1610612738          Boston Celtics          BOS  NBA/BAA  1946  2024   78   
2   1610612751           Brooklyn Nets          BKN  NBA/ABA  1967  2024   57   
3   1610612766       Charlotte Hornets          CHA      NBA  1988  2024   34   
4   1610612741           Chicago Bulls          CHI      NBA  1966  2024   58   
5   1610612739     Cleveland Cavaliers          CLE      NBA  1970  2024   54   
6   1610612742        Dallas Mavericks          DAL      NBA  1980  2024   44   
7   1610612743          Denver Nuggets          DEN  NBA/ABA  1967  2024   57   
8   1610612765         Detroit Pistons          DET  NBA/BAA  1948  2024   76   
9   1610612744   Golden State Warriors          GSW  NBA/BAA  1946  2024   78   
10  1610612745         Houston Rockets          HOU      NBA  1967  2024   57   
11  1610612754          Indi

In [64]:
new_names_teams = ["team_id",
    "franchise",
    "abbvreviation",
    "lg",
    "from",
    "to",
    "years",
    "g",
    "w",
    "l",
    "wl_percentage",
    "playoffs",
    "division",
    "conference",
    "championship"]

print(teams.columns)

teams['from'] = teams['from'].astype(int)
teams['to'] = teams['to'].astype(int)
teams['years'] = teams['years'].astype(int)
teams['g'] = teams['g'].astype(int)
teams['w'] = teams['w'].astype(int)
teams['l'] = teams['l'].astype(int)
teams['wl_percentage'] = teams['wl_percentage'].astype(float)
teams['playoffs'] = teams['playoffs'].astype(int)
teams['division'] = teams['division'].astype(int)
teams['conference'] = teams['conference'].astype(int)
teams['championship'] = teams['championship'].astype(int)
teams['team_id'] = teams['team_id'].astype(int)

teams.columns = new_names_teams
teams.to_csv('teams_new.csv', index=False)

Index(['team_id', 'franchise', 'abbvreviation', 'lg', 'from', 'to', 'years',
       'g', 'w', 'l', 'wl_percentage', 'playoffs', 'division', 'conference',
       'championship'],
      dtype='object')


### Teams Stats: PerGame

In [127]:
team_stats_per_game = pd.read_csv('team_stats_per_game.csv')

actual_teams = pd.read_csv('teams_new.csv')
team_names_active = list(actual_teams['franchise'].unique())

teams = pd.read_csv('teams_active.csv')
for index, row in teams.iterrows():
    from_year  = row['From']
    to_year = row['To']
    two_num = from_year[2:4]
    two_num2 = to_year[5:7]
    # Assign new year to the row
    if (from_year[0] == "1" and from_year[1] == "9" and from_year[2] == "9" and from_year[3] == "9"):
        teams.at[index, 'From'] = int("20{}".format(two_num))
    elif(from_year[0] == "1"):
        teams.at[index, 'From'] = int("19{}".format(two_num))
    elif (from_year[0] == "2"):
        teams.at[index, 'From'] = int("20{}".format(two_num))
        
    if (to_year[0] == "1" and to_year[1] == "9" and to_year[2] == "9" and to_year[3] == "9"):
        teams.at[index, 'To'] = int("20{}".format(two_num2))
    elif (to_year[0] == "1"):
        teams.at[index, 'To'] = int("19{}".format(two_num2))
    elif (to_year[0] == "2"):
        teams.at[index, 'To'] = int("20{}".format(two_num2))
    
#print(teams)
indexes = []
for index, row in actual_teams.iterrows():
    # Find index of the team who is the same in teams
    temp = teams[teams['Franchise'] == row['franchise']]
    temp = temp[temp['From'] == row['from']]
    temp = temp[temp['To'] == row['to']]
    temp = temp[temp['Yrs'] == row['years']]
    temp = temp[temp['G'] == row['g']]
    temp = temp[temp['W'] == row['w']]
    temp = temp[temp['L'] == row['l']]
    # Store index in a list
    indexes.append(temp.index.tolist()[0])
    
name_dict = {}
for i in range(len(indexes)):
    # Get data frame rows between index[i] and index[i+1]
    if (i == len(indexes) - 1):
        temp = teams.iloc[indexes[i]:]
    else:
        temp = teams.iloc[indexes[i]:indexes[i+1]]
    # Get all the unique names in the temp dataframe
    unique_names = list(temp['Franchise'].unique())
    # Add the current nba team name to the dictionary with the unique names
    name_dict[actual_teams.iloc[i]['franchise']] = unique_names
    
for key, value in name_dict.items():
    # If key appears in the value list, remove it
    if (key in value):
        value.remove(key)

# Find all possible names for current active teams
historical_names = []
for key, value in name_dict.items():
    for name in value:
        historical_names.append(name)
for key, value in name_dict.items():
    historical_names.append(key)
    
# Make new dictornary that maps all historical names to the id of the current team
id_dict = {}
actual = pd.read_csv('teams_new.csv')
for index, row in actual.iterrows():
    id_dict[row['franchise']] = row['team_id']
    for name in name_dict[row['franchise']]:
        id_dict[name] = row['team_id']

In [128]:
# Delete all rows where the GP is 0
team_stats_per_game = team_stats_per_game[team_stats_per_game['G'] != 0]
# Keep teams stats if the name is in the historical names list
team_stats_per_game = team_stats_per_game[team_stats_per_game['Team'].isin(historical_names)]
# Map the team name to the team id
team_stats_per_game['TeamID'] = team_stats_per_game['Team'].map(id_dict)
# Make the TeamID column an integer and first column in the dataframe
team_stats_per_game['TeamID'] = team_stats_per_game['TeamID'].astype(int)
columns = team_stats_per_game.columns.tolist()
columns = columns[-1:] + columns[:-1]
team_stats_per_game = team_stats_per_game[columns]
# For the Playoffs column, if the value is NaN, make it Y
team_stats_per_game['Playoffs'].fillna('Y', inplace=True)
# For the Playoffs column, if the value is True, make it Y
team_stats_per_game['Playoffs'] = team_stats_per_game['Playoffs'].replace(True, 'Y')
# For the Playoffs column, if the value is False, make it N
team_stats_per_game['Playoffs'] = team_stats_per_game['Playoffs'].replace(False, 'N')
# For the Playoffs_S column if the value is True, make it Y
team_stats_per_game['Playoffs_S'] = team_stats_per_game['Playoffs_S'].replace(True, 'Y')
# For the Playoffs_S column if the value is False, make it N
team_stats_per_game['Playoffs_S'] = team_stats_per_game['Playoffs_S'].replace(False, 'N')

team_stats_per_game

,TeamID,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,TRB,AST,STL,BLK,TOV,PF,PTS,Year,Playoffs,Playoffs_S
4,1610612744,Philadelphia Warriors,60.0,NaN,25.2,89.7,0.280,NaN,NaN,NaN,...,NaN,5.7,NaN,NaN,NaN,18.0,68.6,1947,Y,N
7,1610612752,New York Knicks,60.0,NaN,24.4,87.6,0.279,NaN,NaN,NaN,...,NaN,7.6,NaN,NaN,NaN,20.3,64.7,1947,Y,N
10,1610612738,Boston Celtics,60.0,NaN,23.3,85.6,0.272,NaN,NaN,NaN,...,NaN,7.8,NaN,NaN,NaN,20.0,60.1,1947,N,N
12,1610612744,Philadelphia Warriors,10.0,NaN,26.5,95.3,0.278,NaN,NaN,NaN,...,NaN,5.2,NaN,NaN,NaN,20.4,75.3,1947,Y,Y
15,1610612752,New York Knicks,5.0,NaN,27.2,100.0,0.272,NaN,NaN,NaN,...,NaN,7.4,NaN,NaN,NaN,20.0,70.6,1947,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2905,1610612763,Memphis Grizzlies,6.0,244.2,38.3,95.3,0.402,12.0,38.5,0.312,...,46.5,23.3,6.7,3.5,13.0,20.5,104.7,2023,Y,Y
2906,1610612755,Philadelphia 76ers,11.0,242.3,36.7,84.0,0.437,12.2,33.9,0.359,...,43.0,19.1,7.6,5.3,12.4,18.7,102.5,2023,Y,Y
2907,1610612752,New York Knicks,11.0,240.0,35.3,81.5,0.433,9.5,32.7,0.292,...,45.4,18.7,7.0,4.5,14.6,19.5,100.1,2023,Y,Y
2908,1610612739,Cleveland Cavaliers,5.0,240.0,35.2,78.4,0.449,10.0,30.6,0.327,...,37.2,22.0,6.8,4.0,15.8,20.4,94.2,2023,Y,Y


In [137]:
team_stats_per_game.dtypes
new_names_team_stats_per_game = ["team_id",
                                 "team",
                                 "g",
                                 "mp",
                                    "fg",
                                    "fga",
                                    "fg_percentage",
                                    "threeP",
                                    "threePA",
                                    "threeP_percentage",
                                    "twoP",
                                    "twoPA",
                                    "twoP_percentage",
                                    "ft",
                                    "fta",
                                    "ft_percentage",
                                    "orb",
                                    "drb",
                                    "trb",
                                    "ast",
                                    "stl",
                                    "blk",
                                    "tov",
                                    "pf",
                                    "pts",
                                    "year",
                                    "playoffs",
                                    "playoffs_s"]

team_stats_per_game.columns = new_names_team_stats_per_game
team_stats_per_game.fillna(-99, inplace=True)
team_stats_per_game['team_id'] = team_stats_per_game['team_id'].astype(int)
team_stats_per_game['g'] = team_stats_per_game['g'].astype(int)
team_stats_per_game['mp'] = team_stats_per_game['mp'].astype(int)

team_stats_per_game.to_csv('team_stats_per_game_new.csv', index=False)
                                    
team_stats_per_game

,team_id,team,g,mp,fg,fga,fg_percentage,threeP,threePA,threeP_percentage,...,trb,ast,stl,blk,tov,pf,pts,year,playoffs,playoffs_s
4,1610612744,Philadelphia Warriors,60,-99,25.2,89.7,0.280,-99.0,-99.0,-99.000,...,-99.0,5.7,-99.0,-99.0,-99.0,18.0,68.6,1947,Y,N
7,1610612752,New York Knicks,60,-99,24.4,87.6,0.279,-99.0,-99.0,-99.000,...,-99.0,7.6,-99.0,-99.0,-99.0,20.3,64.7,1947,Y,N
10,1610612738,Boston Celtics,60,-99,23.3,85.6,0.272,-99.0,-99.0,-99.000,...,-99.0,7.8,-99.0,-99.0,-99.0,20.0,60.1,1947,N,N
12,1610612744,Philadelphia Warriors,10,-99,26.5,95.3,0.278,-99.0,-99.0,-99.000,...,-99.0,5.2,-99.0,-99.0,-99.0,20.4,75.3,1947,Y,Y
15,1610612752,New York Knicks,5,-99,27.2,100.0,0.272,-99.0,-99.0,-99.000,...,-99.0,7.4,-99.0,-99.0,-99.0,20.0,70.6,1947,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2905,1610612763,Memphis Grizzlies,6,244,38.3,95.3,0.402,12.0,38.5,0.312,...,46.5,23.3,6.7,3.5,13.0,20.5,104.7,2023,Y,Y
2906,1610612755,Philadelphia 76ers,11,242,36.7,84.0,0.437,12.2,33.9,0.359,...,43.0,19.1,7.6,5.3,12.4,18.7,102.5,2023,Y,Y
2907,1610612752,New York Knicks,11,240,35.3,81.5,0.433,9.5,32.7,0.292,...,45.4,18.7,7.0,4.5,14.6,19.5,100.1,2023,Y,Y
2908,1610612739,Cleveland Cavaliers,5,240,35.2,78.4,0.449,10.0,30.6,0.327,...,37.2,22.0,6.8,4.0,15.8,20.4,94.2,2023,Y,Y


### Championships

In [129]:
championships = pd.read_csv('championships.csv')
championships = championships[championships['Champion'].isin(historical_names)]
championships['Year'] = championships['Year'].astype(int)
championships['TeamID'] = championships['Champion'].map(id_dict)
championships['TeamID'] = championships['TeamID'].astype(int)
columns = championships.columns.tolist()
columns = columns[-1:] + columns[:-1]
championships = championships[columns]
new_championship_names = ['team_id, lg, year, champion']
championships

,TeamID,Year,Lg,Champion
0,1610612743,2023,NBA,Denver Nuggets
1,1610612744,2022,NBA,Golden State Warriors
2,1610612749,2021,NBA,Milwaukee Bucks
3,1610612747,2020,NBA,Los Angeles Lakers
4,1610612761,2019,NBA,Toronto Raptors
...,...,...,...,...
85,1610612758,1951,NBA,Rochester Royals
86,1610612747,1950,NBA,Minneapolis Lakers
87,1610612747,1949,BAA,Minneapolis Lakers
88,1610612764,1948,BAA,Baltimore Bullets


In [133]:
new_championship_names = ['team_id', 'year', 'lg', 'champion']
championships.columns = new_championship_names
championships.to_csv('championships_new.csv', index=False)

## Forecast

In [155]:
team_forecast = pd.read_csv('team_forecast.csv')
for index, row in team_forecast.iterrows():
    name = _teams.find_teams_by_nickname(row['Team'])
    if (len(name) == 0):
        name = _teams.find_teams_by_full_name(row['Team'])
    team_forecast.at[index, 'team_id'] = name[0]['id']
    team_forecast.at[index, 'team'] = name[0]['full_name']
    
team_forecast['team_id'] = team_forecast['team_id'].astype(int)
team_forecast.drop('Team', axis=1, inplace=True)
columns = team_forecast.columns.tolist()
columns = columns[-1:] + columns[:-1]
columns = columns[-1:] + columns[:-1]
team_forecast = team_forecast[columns]
new_forecast_names = ['team_id', 'team', 'playoff_percentage', 'finals_percentage', 'champ_percentage']
team_forecast.columns = new_forecast_names
team_forecast.to_csv('team_forecast_new.csv', index=False)
